# SIDHforJ im Detail


Dieses Notebook benötigt das Jupyter-Java-Kernel

Dieses Notebook soll die Algorithmik des SIDHforJ im Detail erklären. Dabei stehen vor allem die zahlreichen Optimierungen im Vordergrund. 
Um Redundanz zu vermeiden wurden alle Code-Kommentare entfernt und der Code auf ein Minimum reduziert. Da SIDHforJ keinerlei externe Bibliotheken verwendet, können Sie nach Installation des Jupyter-Java-Kernels den Code in ihrem Browserfenster ausführen. Der eigentliche Vorteil dieses Notebooks ist jedoch die Möglichkeit Formeln, Tabellen, Text und Bilder dazwischen einzufügen.

In [1]:
import java.math.BigInteger;
import java.security.SecureRandom;

Dies sind tatsächlich die einzigen beiden externen Pakete, die SIDHforJ zur Durchführung seiner Kernfunktionalität benötigt. Diese beiden Pakete sind Teil der Java-Standardbibliothek und daher auf jedem Java-fähigen Gerät verfügbar

Jede Berechnung innerhalb des SIDHforJ basiert auf endlichen Feldern. Die Werte der Elemente können dabei, wie bei kryptographischen Anwendungen üblich, sehr groß werden. Da die größtmögliche Java-Primitive **long** lediglich einen Maximalwert von $2^{63}$ _[1]_ besitzt, die Primcharacteristik der NIST-SIDH-Parameterklasse "P503" aber bereits einen Wert von $\approx 2^{502}$ aufweist, muss die Klasse **BigInteger** der Java-Standardbibliothek verwendet werden, die arbiträr große Zahlen speichern kann. 
Galoisfelder und deren quadratische Erweiterungen wurden bereits im Notebook "Grundlagen" besprochen. 
Die folgende Code-Zelle (Jede kohärente Einheit innerhalb des Jupyter-Notebook wird Zelle genannt) implementiert nun die Klasse **GFElement**, welche Elemente von Galoisfeldern repäsentiert.

## Galoisfelder

In [2]:
public class GFElement {
    public BigInteger value;
    public BigInteger p;

    public GFElement(BigInteger value, BigInteger p) {
        this.value = value.mod(p);
        this.p = p;}

    public GFElement(GFElement x) {
        this.value = x.value;
        this.p = x.p;}

    public GFElement(byte[] arr, BigInteger p) {
        this.value = new BigInteger(arr).mod(p);
        this.p = p;}

    public GFElement add(GFElement y) {return new GFElement(value.add(y.value).mod(p), p);}

    public GFElement subtract(GFElement y) {return new GFElement((value.subtract(y.value)).mod(p), p);}
    
    public GFElement multiply(GFElement y) {return new GFElement((value.multiply(y.value)), p);}

    public GFElement invert() {return new GFElement((value.modInverse(p)), p);}

    public GFElement negate() {return new GFElement(p.subtract(this.value), p);}

    public GFElement square() {return new GFElement(this.multiply(this));}

    public GFElement leftShift(int k) {return new GFElement(value.shiftLeft(k), p);}

    public GFElement divideby2() {
        BigInteger val = value;
        if (!isEven()) {val = value.add(p);}
        val = val.shiftRight(1);
        return new GFElement(val, p);}

    public boolean isEven() {
        BigInteger c;
        c = value.and(BigInteger.ONE);
        return c.equals(BigInteger.ZERO);}

    public GFElement swap(GFElement z, BigInteger a) {
        BigInteger t, m;
        m = z.value;
        t = a.negate().and(value.xor(m));
        value = t.xor(value);
        m = t.xor(m);
        return new GFElement(m, p);}

    public byte[] toByteArray() {
        int pLength, vLength, off;
        byte[] arr;
        pLength = (p.bitLength() / 8) + 1;
        vLength = (value.bitLength() / 8) + 1;
        off = pLength - vLength;
        arr = new byte[pLength];
        Arrays.fill(arr, (byte) 0);
        System.arraycopy(value.toByteArray(), 0, arr, off, vLength);
        return arr;}

    public String toString() {return String.format("{ %d } mod %d", this.value, this.p);}
}

Galoisfelder und deren quadratische Erweiterungen wurden bereits im Notebook "Grundlagen" disskutiert. Hier sind lediglich die Optimierungen zur Arithmetik auf endlichen Feldern relevant, die SIDHforJ implementiert.
1. Multiplikation und Division durch 2:
    Wenn ein Element mit 2 mulipliziert/dividiert werden soll, macht sich SIDHforJ die Binärdarstellung des Wertes zu nutze. Dazu ein Beispiel: 
Wir nehmen an, wir wollen die Zahl $44$ mit 2 multiplizieren. Die Binärdarstellung dieser Zahl lautet: $binary(23) = 00010111$. Wenn wir diese Binärzahl nun um ein Stelle nach links     verschieben und eine Null am Ende anhängen, erhalten wir die Binärzahl: $00101110$, wobei gilt $decimal(00101110) = 46$. Mit der gleichen Methode zur Bitweisen               Verschiebung nach rechts, kann eine Zahl durch 2 geteilt werden. Dies ist eine extrem performate Weise der Multiplikation.
2. Konvertierung in Byte-Arrays:
   Sowohl im QrTOR-Projekt als auch im SIDHforJ spielen Byte-Arrays eine große Rolle. So können alle relevanten Datenstrukturen innerhalb SIDHforJ in ByteArrays transformiert werden.   Dies hat den Vorteil, dass SIDHforJ so zum Beispiel nahtlos an die **javax.crypto**-API integriert werden kann (wie in QrTOR-Core durchgeführt). Die Methode **toByteArray** implementiert daher eine solche Konvertierung, wobei am Ende lediglich der Wert des Elements umgewandelt wird.

Da innerhalb des SIDHforJ alle Berechnungen in der quadratischen Erweiterung $GF(p^2)$ stattfinden, muss auch eine solche Klasse **GF2Element** implementiert werden. Diese baut direkt auf der voran definierten Klasse **GFElement** auf, wobei **GF2Element** jeweils 2 Instanzen der KLasse **GFElement** speichert

In [3]:
public class GF2Element {
    public GFElement x0;
    public GFElement x1;

    public GF2Element(GF2Element x) {
        this.x0 = x.x0;
        this.x1 = x.x1;}

    public GF2Element(GFElement x0, GFElement x1) {
        if (x0.p.equals(x1.p)) {
            this.x0 = x0;
            this.x1 = x1;
        } else {
            System.out.println("Invalid GF(p^2) Definition");
            System.exit(-1);}}
    
    public GF2Element(BigInteger x0, BigInteger x1, BigInteger p) {
        this.x0 = new GFElement(x0, p);
        this.x1 = new GFElement(x1, p); }

    public static GF2Element ONE(BigInteger p) {
        return new GF2Element(new GFElement(BigInteger.ONE, p), new GFElement(BigInteger.ZERO, p));}

    public static GF2Element ZERO(BigInteger p) {
        return new GF2Element(new GFElement(BigInteger.ZERO, p), new GFElement(BigInteger.ZERO, p));}

    public GF2Element add(GF2Element x) {
        GFElement a0 = new GFElement(this.x0.add(x.x0));
        GFElement a1 = new GFElement(this.x1.add(x.x1));
        return new GF2Element(a0, a1);}

    public GF2Element subtract(GF2Element x) {
        GFElement a0 = new GFElement(this.x0.subtract(x.x0));
        GFElement a1 = new GFElement(this.x1.subtract(x.x1));
        return new GF2Element(a0, a1);}

    public GF2Element multiply(GF2Element y) {
        GFElement t1, t2, c0, c1, r0, r1;
        t1 = x0.multiply(y.x0);
        t2 = x1.multiply(y.x1);
        c0 = t1.subtract(t2);
        t1 = t1.add(t2);
        t2 = x0.add(x1);
        r1 = y.x0.add(y.x1);
        r1 = r1.multiply(t2);
        r1 = r1.subtract(t1);
        r0 = c0;
        return new GF2Element(r0, r1);}

    public GF2Element invert() {
        GFElement t0, a0, a1;
        t0 = this.x0.square();
        t0 = t0.add(x1.square());
        t0 = t0.invert();
        a1 = this.x1.negate();
        a0 = this.x0.multiply(t0);
        a1 = a1.multiply(t0);
        return new GF2Element(a0, a1);}

    public GF2Element shift(int k) {
        return new GF2Element(x0.leftShift(k), x1.leftShift(k));}

    public GF2Element divideby2() {
        return new GF2Element(x0.divideby2(), x1.divideby2());}

    public GF2Element swapWith(GF2Element z, BigInteger op) {
        GFElement a0, a1;

        a0 = this.x0.swap(z.x0, op);
        a1 = this.x1.swap(z.x1, op);

        return new GF2Element(a0, a1);}

    public GF2Element square() {
        GFElement t1, t2, t3, a0, a1;

        t1 = x0.add(x1);
        t2 = x0.subtract(x1);
        t3 = x0.leftShift(1);

        a0 = t1.multiply(t2);
        a1 = x1.multiply(t3);

        return new GF2Element(a0, a1);}

    public static GF2Element[] threeInverts(GF2Element z0, GF2Element z1, GF2Element z2) {
        GF2Element t0;
        GF2Element res[];
        res = new GF2Element[3];
        t0 = z0.multiply(z1);
        res[1] = t0.multiply(z2);
        res[2] = res[1].invert();
        res[1] = res[2].multiply(z2);
        res[0] = res[1].multiply(z1);
        res[1] = res[1].multiply(z0);
        res[2] = res[2].multiply(t0);
        return res;}

    public byte[] toByteArray() {
        int pLength;
        byte[] arr;
        pLength = (x0.p.bitLength() / 8) + 1;
        arr = new byte[pLength * 2];
        System.arraycopy(x0.toByteArray(), 0, arr, 0, pLength);
        System.arraycopy(x1.toByteArray(), 0, arr, pLength, pLength);
        return arr;}

    public static GF2Element fromByteArray(byte[] arr, BigInteger p) {
        GFElement x0, x1;
        int pLength;
        pLength = arr.length / 2;
        x0 = new GFElement(Arrays.copyOfRange(arr, 0, pLength), p);
        x1 = new GFElement(Arrays.copyOfRange(arr, pLength, 2*pLength), p);
        return new GF2Element(x0, x1);}

    public String toString() {return String.format("%d + %d * i {%d}", x0.value, x1.value, x0.p);}

    public boolean equals(GF2Element a) {
        if (this.x0.value.equals(a.x0.value) && (this.x1.value.equals(a.x1.value))
            && this.x0.p.equals(a.x0.p)) {
            return true;
        } else {
            return false;}}
}

Auch hier wird die grundlegenden arithmetischen Funktionalität nicht erklärt, sondern lediglich auf die diversen, in dieser Klasse eingebauten Optimierungen eingegangen.
1. Quadrierung eines Elements $e^2 = (x_0 + i * x_1)^2 = ((x_0 + x_1)(x_0 - x_1) + i * 2x_0x_1)$ 
    Die Quadrierung eines Elements der quadratischen Erweiterung wird ebenfalls durch die obig vorgestelle Binärverschiebung beschleunigt
2. Berechnung der multiplikativen Inverse 
    Die kostspielige Berechnung der Inverse eines Elements wird mittels des erweiterten euklidschen Algorithmus durchgeführt.
3. Konvertierung in/aus Byte-Arrays 
    Die Klasse stellt zudem aus den oben genannten Gründen Methoden zur De-/Serialisierung aus/in Byte-Arrays bereit.

Hierzu einige Beispiele mit dem Element $2 + 4i \in GF(13^2)$:

In [4]:
GF2Element e = new GF2Element(BigInteger.valueOf(2), BigInteger.valueOf(4), BigInteger.valueOf(13));

System.out.println(e.square());

System.out.println(e.invert());

byte[] arr = e.toByteArray();
System.out.println(Arrays.toString(arr));

System.out.println(GF2Element.fromByteArray(arr, BigInteger.valueOf(13)))

1 + 3 * i {13}
4 + 5 * i {13}
[2, 4]
2 + 4 * i {13}


Wie schon bei alleiniger Betrachtung des obigen Codes vermerkt werden kann, ist, dass die Methode **invert()** die höchste Komplexität besitzt und aus Performancegründen möglichst vermieden werden sollten. Viele im Folgenden besprochenen, optimierten Algorithmen sind auf eine möglichst sparsame Verwendung dieser Funktion ausgelegt. Diese Methode wurde bereits im Notebook "Grundlagen" im Kapitel "Mongomerykurven" besprochen

## Montgomerypunkte

Wie bereits im Notebook "Grundlagen" ausführlich dargelegt, werden in der Praxis zumeist elliptische Kurven in Montgomeryform verwendet und deren Punkte in Projektivkoordinatenform angegeben. Jeder Punkt auf einer Montgomerykurve wird daher durch SIDHforJ mit folgender Form implementiert: $$P_{Montgomery} = (a : z) \ ; \ a, z \in GF(p^2)$$ 
In der Praxis werden die beiden Projektivkoordinaten wiederum durch jeweils eine Instanz der Klasse **GF2Element** repräsentiert.

In [5]:
public class MontgomeryPoint {
    public GF2Element x;
    public GF2Element z;

    public MontgomeryPoint(GF2Element x, GF2Element z) {
        this.x = x;
        this.z = z;}

    public void setX(GF2Element x) {this.x = x;}

    public void setZ() {this.z = z;}
    
    public MontgomeryPoint swapPoints(MontgomeryPoint q, BigInteger op) {
        GF2Element qx, qz;
        qx = this.x.swapWith(q.x, op);
        qz = this.z.swapWith(q.z, op);
        return new MontgomeryPoint(qx, qz);}}

So würde der Montgomerypunkt $P = ((4 + 2i) : (8 - 9i))$ folgendermaßen initialisiert werden:

In [6]:
GF2Element P_x = new GF2Element(BigInteger.valueOf(4), BigInteger.valueOf(2), BigInteger.valueOf(13));
GF2Element P_z = new GF2Element(BigInteger.valueOf(8), BigInteger.valueOf(-9), BigInteger.valueOf(13));
MontgomeryPoint P = new MontgomeryPoint(P_x, P_z);

## Montgomerykurven

Dazu kommt eine weitere Besonderheit, um die Performance zu steigern. Im Gegensatz zur gängigen Praxis, ledigleich die Punkte auf einer Kurve in Projektivkoordinaten anzugeben, wird auch die Kurve selbst (bzw. die Kurvenparameter) in Projektivform angegeben _[2]_. Daher gilt für jede Montgomerykurve $E$ in SIDHforJ: 
$$E_{Montgomery}: by^2 = x^3 + ax^2 + x$$ </br> $$(A : B : C) \in \mathbb{P}^2 \ , \ a = \frac{A}{C} \ , \ b = \frac{B}{C} $$  </br> $$E_{SIDHforJ}: By^2 = Cx^3 + Ax^2 + Cx; \ B=1$$ </br>

Die folgende Klasse beschreibt nun eine Montgomerykurve in ihrer oben genannten Projektivform. Dabei speichert diese grundsätzlich lediglich die beiden Parameter $A$ und $C$. 
In den Methoden **doubleP**, **tripleP** und auch in den späteren Isogenieberechnungen benötigen wir oftmals gleiche Werte. So wird zum Beispiel in den ersten beiden Methoden der Wert $a + 2c$ verwendet. Um die Berechnungen zu beschleunigen werden daher einige Variablen (die ausschließlich von den Kurvenparameter $(a : c)$ abhängig sind) im Voraus berechnet und gespeichert. 
Die Klasse implementiert zudem bereits bekannte Funktionen zur Berechnung der Skalarmultiplikation. Selbstverständlich mussten diese Formeln an die Projektivform des Kurvenparameters angepasst werden _[2]_.

In [7]:
public class MontgomeryCurve {

    public GF2Element a, c;
    public GF2Element a24, c4;
    public GF2Element aP2c, aM2c;

    public MontgomeryCurve() {
        this.a = GF2Element.ZERO(BigInteger.TWO);
        this.c = GF2Element.ONE(BigInteger.TWO);}

    public MontgomeryCurve(GF2Element a, GF2Element c) {
        this.a = a;
        this.c = c;}

    public MontgomeryCurve (MontgomeryCurve curve) {
        this.a = curve.a;
        this.c = curve.c;}

    public MontgomeryCurve(BigInteger p) {
        this.a = GF2Element.ZERO(p);
        this.c = GF2Element.ONE(p);}

    public void setA24(GF2Element a24) {
        this.a24 = a24;}

    public void setC4(GF2Element c4) {
        this.c4 = c4;}

    public void setaP2c(GF2Element aP2c) {
        this.aP2c = aP2c;}

    public void setaM2c(GF2Element aM2c) {
        this.aM2c = aM2c;}

    public void calculateA24(BigInteger p) {
        a24 = GF2Element.ONE(p);
        a24 = a24.shift(1);
        a24 = a24.add(a);
        a24 = a24.divideby2();
        a24 = a24.divideby2();}

    public void calculatePM() {
        GF2Element c2 = this.c.shift(1);
        aP2c = this.a.add(c2);
        aM2c = this.a.subtract(c2);}

    public void calculateC4() {
        c4 = this.c.shift(2);}

    public void calculateAC(int order) {
        if (order == 3) {
            a = aP2c.add(aM2c);
            a = a.shift(1);
            c = aP2c.subtract(aM2c);
        } else {
            c = c4.divideby2();
            a = aP2c.subtract(c);
            c = c.divideby2();}}

    public MontgomeryPoint doubleP(MontgomeryPoint p) {
        GF2Element t0, t1, qx, qz, px, pz;
        px = p.x;
        pz = p.z;
        t0 = px.subtract(pz);
        t1 = px.add(pz);
        t0 = t0.square();
        t1 = t1.square();
        // Hier findet beispielsweise der vorab berechnete Wert: c * 4 Anwendung
        qz = c4.multiply(t0);
        qx = t1.multiply(qz);
        t1 = t1.subtract(t0);
        t0 = aP2c.multiply(t1);
        qz = qz.add(t0);
        qz = qz.multiply(t1);
        return new MontgomeryPoint(qx, qz);}

    public MontgomeryPoint tripleP(MontgomeryPoint p) {
        GF2Element t0, t1, t2, t3, t4, t5, t6, px, pz, qx, qz;
        px = p.x;
        pz = p.z;
        t0 = px.subtract(pz);
        t2 = t0.square();
        t1 = px.add(pz);
        t3 = t1.square();
        t4 = px.shift(1);
        t0 = pz.shift(1);
        t1 = t4.square();
        t1 = t1.subtract(t3);
        t1 = t1.subtract(t2);
        t5 = t3.multiply(aP2c);
        t3 = t3.multiply(t5);
        t6 = t2.multiply(aM2c);
        t2 = t2.multiply(t6);
        t3 = t2.subtract(t3);
        t2 = t5.subtract(t6);
        t1 = t1.multiply(t2);
        t2 = t1.add(t3);
        t2 = t2.square();
        qx = t4.multiply(t2);
        t1 = t3.subtract(t1);
        t1 = t1.square();
        qz = t0.multiply(t1);
        return new MontgomeryPoint(qx, qz);}

    public MontgomeryPoint multiply2X(MontgomeryPoint p, int k) {
        MontgomeryPoint q = p;
        int i;
        for (i = 0; i < k; i++) {
            q = doubleP(q);}
        return q;
    }

    public MontgomeryPoint multiply3X(MontgomeryPoint p, int k) {
        MontgomeryPoint q = p;
        int i;
        for (i = 0; i < k; i++) {
            q = tripleP(q);}
        return q;
    }

    public MontgomeryPoint[] doubleAdd(MontgomeryPoint p, MontgomeryPoint q, GF2Element xpq) {
        GF2Element t0, t1, t2, qx, qz, px, pz;
        MontgomeryPoint pq[];
        px = new GF2Element(p.x);
        pz = new GF2Element(p.z);
        qx = new GF2Element(q.x);
        qz = new GF2Element(q.z);
        t0 = px.add(pz);
        t1 = px.subtract(pz);
        px = t0.square();
        t2 = qx.subtract(qz);
        qx = qx.add(qz);
        t0 = t0.multiply(t2);
        pz = t1.square();
        t1 = t1.multiply(qx);
        t2 = px.subtract(pz);
        px = px.multiply(pz);
        qx = t2.multiply(a24);
        qz = t0.subtract(t1);
        pz = pz.add(qx);
        qx = t0.add(t1);
        pz = pz.multiply(t2);
        qz = qz.square();
        qx = qx.square();
        qz = qz.multiply(xpq);
        pq = new MontgomeryPoint[2];
        pq[0] = new MontgomeryPoint(px, pz);
        pq[1] = new MontgomeryPoint(qx, qz);
        return pq;}

    public MontgomeryPoint ladder3pt(GF2Element xp, GF2Element xq, GF2Element xpq, BigInteger m, int obits, BigInteger p) {
        MontgomeryPoint rs[], r;
        int i;
        BigInteger swap, bit, prevbit = BigInteger.ZERO;
        GF2Element xval;
        rs = new MontgomeryPoint[2];
        rs[0] = new MontgomeryPoint(xq, GF2Element.ONE(p));
        rs[1] = new MontgomeryPoint(xpq, GF2Element.ONE(p));
        r = new MontgomeryPoint(xp, GF2Element.ONE(p));
        for (i = 0; i < obits; i++) {
            bit = m.testBit(i) ? BigInteger.ONE : BigInteger.ZERO;
            swap = bit.xor(prevbit);
            prevbit = bit;
            r = rs[1].swapPoints(r, swap);
            rs = doubleAdd(rs[0], rs[1], r.x);
            rs[1].setX(rs[1].x.multiply(r.z));}
        return r;}
    
    public GF2Element jInv() {
        GF2Element t0, t1, jinv;
        jinv = a.square();
        t1 = c.square();
        t0 = t1.shift(1);
        t0 = jinv.subtract(t0);
        t0 = t0.subtract(t1);
        jinv = t0.subtract(t1);
        t1 = t1.square();
        jinv = jinv.multiply(t1);
        t0 = t0.shift(2);
        t1 = t0.square();
        t0 = t0.multiply(t1);
        t0 = t0.shift(2);
        jinv = jinv.invert();
        jinv = jinv.multiply(t0);
        return jinv;}

    public static GF2Element recoverFromPoints(GF2Element px, GF2Element qx, GF2Element dx) {
        GF2Element t0, t1, ra;
        t1 = px.add(qx);
        t0 = px.multiply(qx);
        ra = dx.multiply(t1);
        ra = ra.add(t0);
        t0 = t0.multiply(dx);
        ra = ra.subtract(GF2Element.ONE(px.x0.p));
        t0 = t0.shift(2);
        t1 = t1.add(dx);
        ra = ra.square();
        t0 = t0.invert();
        ra = ra.multiply(t0);
        ra = ra.subtract(t1);
        return ra;}
}

Zunächst werden Funktionen zur Punktdopplung $doubleP((x : z)) = [2](x : z)$ und zur Punktverdreifachung $tripleP((x : z)) = [3](x : z)$ implementiert. 
Die explizite Formel der Punktdopplung lautet zum Beispiel wie folgt:

Punktdopplung $[2]P = [2](x : z)$: $$[2](x : z) = (x_q : z_q) \ ; \ x_q = 4c(x - z)^2 (x + z)^2 \ ; \ z_q = 4cx^2+4axz+4xz+4cz^2$$
Die restlichen Algorithmen können selbstverständlich auch durch eine solche explizite Formel notiert werden. Da dieses Notebook die Optimierungen so kompakt wie möglich zu erklären versucht, wird hier auf weitere Formeln verzichtet. Viel interessanter ist es ohnehin die Eingaben, Ausgaben und notwendigen Operationen dieser Algorithmen aus einer mathematischen Perspektive zu betrachten _[2]_:

| Algorithmus | Eingabe                            | Ausgabe         | Addition | Subtraktion | Multiplikation | Inversion |
|-------------|------------------------------------|-----------------|----------|-------------|----------------|-----------|
| doubleP     | $$(P, a + 2c, 4c)$$                | $$[2]P$$        | 4        | 2           | 4              | 0         |
| tripleP     | $$(P, a + 2c, 4c)$$                | $$[3]P$$        | 8        | 4           | 8              | 0         |
| doubleAdd   | $$(P, Q, x_{P-Q}, \frac{a+2}{4})$$ | $$[2]P, Q + P$$ | 8        | 4           | 6              | 0         |
| ladder3pt   | $$(P, Q, P-Q, a, m)$$              | $$P+[m]Q$$      | 14n+3    | 6n          | 9n             | 0         |

Wie in der Tabelle zu sehen berechnet die Funktion **doubleAdd** zwei Punkte: Die Sklaramultiplikation $2[P]$ und die Punktaddition $P + Q$. Hier handelt es sich zum Beispiel um einen hochoptimierten Algorithmus, der deutlich (nahezu doppelt) schneller ist, als wenn die Skalarmultiplikation und die Punktaddition hintereinander berechnet werden würden.  
Der vierte Algorithmus **ladder3pt** berechnet mithilfe der berühmten Montgomeryladder eine Linearkombination aus zwei gegebenen Punkten. Dieser Algorithmus wird in SIDHforJ exzessiv für die Berechnung von Isogeniegeneratorenpunkten verwendet. Die vier in der Tabelle gezeigten Algorithmen sind tatsächlich für die Implementierung eines SIDH-Protokolls ausreichend.
Es ist zu beachten, dass alle bisher vorgestellten und implementierten Algorithmen durchaus auch in herkömmlichen, auf elliptischen Kurven basierenden, kryptographischen Systemen zum Einsatz kommen könnten.

## Isogenieberechnungen

Wir erreichen nun das Herzstück des SIDHforJ: Die Algorithmen zur Isogenieberechnung. Dazu folgt zunächst die Theorie hinter der Optimierung: 
Eine SIDHforJ-Isogenie speichert grundsätzlich 2 Strukturen. Die Bildkurve die durch die Isogenie generiert wird und die Isogenie selbst. Um eine Isogenie $\phi$ aus einer Montgomerykurve $E_0$ zu berechnen, muss zunächst der Isogeniegeneratorpunkt $R$ ausgewählt werden. Danach wird eine neue Instanz der Klasse **Isogeny** aufgerufen, wobei die Urbild-Kurve $E_0$ in den Konstruktor übergeben wird. Die eigentliche Berechnung erfolgt nach dem Aufruf der Methode **getIsogeny** wobei diese als Parameter den Isogeniegeneratorpunkt $R$ erhält. Die **Isogeny**-Instanz berechnet zunächst die Isogeniefunktion (bzw. deren Koeffizienten) und danach die Kurvenparameter der Bildkurve.  
In SIDHforJ werden ausschließlich Isogenien vom Grad $3$ oder vom Grad $4$ verwendet. Daher implementiert SIDHforJ zwei Klassen: **ThreeIsogeny** für Isogenien des Grads $3$ und **FourIsogeny** für Isogenien des Grads $4$. Dies hat den Vorteil, dass die hocheffizienten, in _[1]_ beschriebenen Algorithmen für Isogenieberechnung dieses speziellen Grads implementiert werden können.
Im Folgenden werden die expliziten Formeln für die optimierten Algorithmen notiert:
1. Isogenien von Grad $3$ \
    1.1 Berechnung der Bildkurve:
    $$getIsogeny((x_g : z_g)) = (z_g^4 + 18x_g^2z_g^2-27x_g^4 : 4x_gz_g^3) = (a_{Bildkurve} : c_{Bildkurve})$$
    1.2 Berechnung des Bild eines Punktes auf $E_0$:
    $$getImageByIsogeny((x : z)) = (x(x_gz - z_gz)^2 : z(z_gx - x_gz)^2) = (x_{Bild} : Z_{Bild})$$
    
2. Isogenien vom Grad $4$ \
    2.1 Berechnung der Bildkurve:
    $$getIsogeny((x_g : z_g)) = (2(2x_g^4-z_g^4) : z_g^4) = (a_{Bildkurve} : c_{Bildkurve})$$
    2.2 Berechnung des Bild eines Punktes auf $E_0$:
    $$getImageByIsogeny((x : z)) = (x(2x_gz_gz - x(x_g^2+z_g^2))(x_gx - z_gz)^2 : z(2x_gz_gx - z(x_g^2+z_g^2))(z_gx - x_gz)^2)$$
    
Wie Sie sehen, beinhalten die expliziten Formeln des Algorithmus **getImageByIsogeny** jeweils wieder die Projektivkoordinaten $x_g$ und $z_g$ des Isogeniegeneratorenpunktes. In der Implementierung des SIDHforJ werden die, von den Koordinaten des Isogeniegenerators abhängigen, Koeffizienten in einer Liste gespeichert. Man hätte alternativ auch einfach den Isogeniegeneratorpunkt in der Funktion **getIsogeny** speichern und dessen Koordinaten bei Aufrufen der Funktion **getImageByIsogeny** wiederverwenden können, doch diese Lösung schien mir eleganter. 
Wie fassen nun nochmals den Ablauf der Isogenieberechnung mit SIDHforJ zusammen:
1. Initialisierung einer Instanz der Klasse **MontgomeryCurve**
2. Selektierung und Initialisierung eines **MontgomeryPoint** auf der Kurve. Dieser Punkt **MUSS** entweder die Ordnung $3$ oder $4$ besitzen
3. Initialisierung einer Instanz der Klasse **ThreeIsogeny** oder **FourIsogeny** (dies hängt von der Ordnung des Punkts ab). Übergeben der initialen **MontgomeryCurve** in den Konstruktor
4. Aufrufen der Methode **getIsogeny** mit dem ausgewählten Punkt der spezifischen Ordnung
5. Die Klasse **ThreeIsogeny** oder **FourIsogeny** repräsentiert nun die Isogenie **UND** die Bildkurve. Da beide Klassen von der Klasse **MontgomeryCurve** erben, könnten also hier bereits Skalarmultiplikationen auf der Bildkurve durchgeführt werden
5. Aufrufen der Methode **getImageByIsogeny** mit einen Punkt der initialen Kurve als Argument. Ergebnis ist dessen Bildpunkt auf der Bildkurve

In [8]:
public class ThreeIsogeny extends MontgomeryCurve {

    GF2Element coefficients[];

    public ThreeIsogeny(MontgomeryCurve curve) {
        coefficients = new GF2Element[2];
        this.a = curve.a;
        this.c = curve.c;
        this.a24 = curve.a24;
        this.c4 = curve.c4;
        this.aP2c = curve.aP2c;
        this.aM2c = curve.aM2c;}

    public ThreeIsogeny(GF2Element a, BigInteger p) {
        coefficients = new GF2Element[2];
        this.a = a;
        this.c = GF2Element.ONE(p);}
    
    public void getIsogeny(MontgomeryPoint p) {
        GF2Element t0, t1, t2, t3, t4, px, pz;
        px = p.x;
        pz = p.z;
        coefficients[0] = px.subtract(pz);
        t0 = coefficients[0].square();
        coefficients[1] = px.add(pz);
        t1 = coefficients[1].square();
        t2 = t0.add(t1);
        t3 = coefficients[0].add(coefficients[1]);
        t3 = t3.square();
        t3 = t3.subtract(t2);
        t2 = t1.add(t3);
        t3 = t3.add(t0);
        t4 = t0.add(t3);
        t4 = t4.shift(1);
        t4 = t4.add(t1);
        aM2c = t2.multiply(t4);
        t4 = t1.add(t2);
        t4 = t4.shift(1);
        t4 = t4.add(t0);
        t4 = t4.multiply(t3);
        t0 = t4.subtract(aM2c);
        aP2c = t0.add(aM2c);}

    public MontgomeryPoint imageByIsogeny(MontgomeryPoint q) {
        GF2Element t0, t1, t2, qx, qz;
        qx = q.x;
        qz = q.z;
        t0 = qx.add(qz);
        t1 = qx.subtract(qz);
        t0 = t0.multiply(coefficients[0]);
        t1 = t1.multiply(coefficients[1]);
        t2 = t1.add(t0);
        t0 = t1.subtract(t0);
        t2 = t2.square();
        t0 = t0.square();
        qx = qx.multiply(t2);
        qz = qz.multiply(t0);
        return new MontgomeryPoint(qx, qz);}
}

In [9]:
public class FourIsogeny extends MontgomeryCurve {
    GF2Element coefficients[];

    public FourIsogeny(GF2Element a, BigInteger p) {
        coefficients = new GF2Element[3];
        this.a = a;
        this.c = GF2Element.ONE(p);}

    public FourIsogeny(MontgomeryCurve curve) {
        coefficients = new GF2Element[3];
        this.a = curve.a;
        this.c = curve.c;
        this.a24 = curve.a24;
        this.c4 = curve.c4;
        this.aP2c = curve.aP2c;
        this.aM2c = curve.aM2c;}

    public void getIsogeny(MontgomeryPoint p) {
        GF2Element px, pz;
        px = p.x;
        pz = p.z;
        coefficients[1] = px.subtract(pz);
        coefficients[2] = px.add(pz);
        coefficients[0] = pz.square();
        coefficients[0] = coefficients[0].shift(1);
        c4 = coefficients[0].square();
        coefficients[0] = coefficients[0].shift(1);
        aP2c = px.square();
        aP2c = aP2c.shift(1);
        aP2c = aP2c.square();}

    public MontgomeryPoint imageByIsogeny(MontgomeryPoint p) {
        GF2Element t0, t1, px, pz;
        px = p.x;
        pz = p.z;
        t0 = px.add(pz);
        t1 = px.subtract(pz);
        px = t0.multiply(coefficients[1]);
        pz = t1.multiply(coefficients[2]);
        t0 = t0.multiply(t1);
        t0 = t0.multiply(coefficients[0]);
        t1 = px.add(pz);
        pz = px.subtract(pz);
        t1 = t1.square();
        pz = pz.square();
        px = t0.add(t1);
        t0 = pz.subtract(t0);
        px = px.multiply(t1);
        pz = pz.multiply(t0);
        return new MontgomeryPoint(px, pz);}
}

Wie haben nun das Ende des mathematischen Teils erreicht. Innerhalb der originalen Implementierung wären alle diese Funktionalitäten innerhalb des "math"-Pakets zu finden. 
Im Folgenden wollen wir nun die mathematischen Strukturen in kryptographische Algorithmen überführen. Sprechen wir dazu zunächst über die öffentlichen Parameter, die sich die beiden Parteien teilen. Im Folgenden werden diese Parteien traditionellerweise als Alice und Bob bezeichnet. Dies dient lediglich der Veranschaulichung.

## Öffentliche Parameter

In jedem asymmetischen Kryptosystem müssen vor der Durchführung der eigentlichen Algorithmik öffentliche Parameter an beide Parteien übermittelt werden. Alice und Bob bekommen dabei dieselben Parameter zugeteilt. Für den SIDH werden folgende Parameter benötigt:
1. Der Grad der verwendeten Isogenien 
   Alice und Bob verwenden unterschiedliche Isogeniegrade, um die kryptographische Komplexität zu erhöhen _[4]_. In SIDHforJ besitzt jede Isogenie einen Grad der Form $3^n$ oder der Form $4^n$. Die Partei, die innerhalb des Codes als Alice bezeichnet wird verwendet dabei immer Isogeniegrade der Form $4^n$, Bob schlussendlich Grade der Form $3^n$. Wie man erkennen kann, werden die tatsächlichen Isogeniegrade einzig durch den Parameter $n$ bestimmt. Dieser Parameter ist Teil der öffentlichen Parameter, wobei jeweils ein Exponent für Alice und ein von diesem unterschiedlicher Exponent für Bob spezifiziert werden muss. Im Folgenden werden diese Exponenten als $e_A$ für Alice und $e_B$ für Bob bezeichnet. Alice verwendet also Isogenien des Grads $4^{e_A}$ und Bob Isogenien des Grads $3^{e_B}$
   
2. Das Galoisfeld $GF(p)$ und dessen quadratische Erweiterung $GF(p^2)$ 
    Ein Galoisfeld (und damit auch seine quadratische Erweiterung) wird einzig durch seine Primcharactersitik $p$ definiert. Die Primzahl $p$ ist dabei ein weiterer Bestandteil der öffentlichen Parameter.
    Es ist gängige Praxis innerhalb des SIDH, dass die Isogeniegrade $4^{e_A}$ und $3^{e_B}$ so gewählt werden, dass deren Multiplikation  $4^{e_A} * 3^{e_B} - 1$ _[3]_ wiederum eine Primzahl ergibt. Diese Primzahl wird dann als Charakteristik des endlichen Definitionsfelds genutzt.

3. Die initiale elliptische Kurve 
    Die initiale elliptische Kurve (bzw. deren Kurvenparameter) sind ebenfalls Bestandteil der öffentlichen Parameter.
4. Die Generatorenpunkte 
    Um einerseits die Isognien zu generieren und andererseit bereits vorab berechnete Punkte auf der initialen Kurve zu besitzen, werden zuätzlich jeweils 2 Punkte auf der initialen elliptischen Kurve gewählt: ${P_A, Q_A}$ und ${P_B, Q_B}$. Damit diese Punkte in der Lage sind, Isogenien des Grads $3^{e_B}$ bzw. $4^{e_A}$ zu berechnen, müssen diese die jeweilige Ordnung von $3^{e_B}$, bzw. $4^{e_A}$ besitzen:
    $$order(P_A, Q_A) = 4$$
    $$order(P_B, Q_B) = 3$$
    Es wird zusätzlich noch jeweils ein dritter Punkt $D$ als öffentlicher Parameter hinzugefügt, der durch die Formel $D = P - Q$ _[4]_ berechnet werden kann. Dies ist für die **doubleAdd** Methode notwendig
5. Die Strategie zur Generierung der Torsionsgruppe 
    Um den hocheffizienten Algorithmus zur Isogenieberechnung einzusetzen, welcher im Notbook "Theorie" vorgestellt wurde, werden zusätzlich noch die beiden Strategielisten für Alice und Bob benötigt

Die folgende Klasse implementiert nun eine Abwandlung der in SIDHforJ integrierten **PublicParameters** Klasse

In [10]:
public class PublicParameters {

    public BigInteger f;
    public BigInteger lA, lB;
    public int eA, eB;
    public BigInteger p;
    public BigInteger orderA, orderB;
    public int primeLength;
    public int orderLengthA, orderLengthB;
    public GF2Element aPx, aQx, aDx, bPx, bQx, bDx;
    public int maxA, maxB;
    public int maxIntPointsA, maxIntPointsB;
    public int[] splitsA, splitsB;
    public int pubKeyLength;
    public MontgomeryCurve curve;

    public PublicParameters(BigInteger f, BigInteger lA, BigInteger lB, int eA, int eB, GF2Element[] generatorsA, GF2Element[] generatorsB, int[] splitsA, int[] splitsB, int maxPointsA, int maxPointsB) {
        this.f = f;
        this.lA = lA;
        this.lB = lB;
        this.eA = eA;
        this.eB = eB;
        this.p = (((lA.pow(eA)).multiply(lB.pow(eB))).multiply(f)).subtract(BigInteger.ONE);

        this.aPx = generatorsA[0];
        this.aQx = generatorsA[1];
        this.aDx = generatorsA[2];
        this.bPx = generatorsB[0];
        this.bQx = generatorsB[1];
        this.bDx = generatorsB[2];

        this.orderA = lA.pow(eA);
        this.orderB = lB.pow(eB);
        this.splitsA = splitsA;
        this.splitsB = splitsB;
        this.maxA = this.splitsA.length + 1;
        this.maxB = this.splitsB.length + 1;
        this.orderLengthA = orderA.bitLength();
        this.orderLengthB = orderB.bitLength();
        this.maxIntPointsA = maxPointsA;
        this.maxIntPointsB = maxPointsB;
        this.primeLength = (this.p.bitLength() / 8) + 1;
        this.pubKeyLength = this.primeLength * 6;
        this.curve = new MontgomeryCurve(p);}
    
    public GF2Element[] getGeneratorA() {
        return new GF2Element[] {aPx, aQx, aDx};
    }

    public GF2Element[] getGeneratorB() {
        return new GF2Element[] {bPx, bQx, bDx};
    }
} 

## Berechnung der öffentlichen Schlüssel

Wir beginnen zunächst mit der Funktion, die später Alice's öffentlichen Schlüssel berechnen soll. Diese Funktion wurde durch Kommentare in 4 Sektionen (A, B, C, D) getrennt.

In [11]:
public GF2Element[] generatePubA(BigInteger priv, PublicParameters params) {
    // Sektion A
    MontgomeryCurve curve;
    BigInteger p;
    FourIsogeny fIso;
    GF2Element genA[], genB[], inverses[], phiPX, phiQX, phiDX;
    MontgomeryPoint phiP, phiQ, phiD, r;
    MontgomeryPoint points[];
    int maxIntPointsA, maxA, row, index = 0, npts = 0, obits, ii = 0, m, i;
    int splitsA[], pointsIndx[];

    curve = params.curve;
    p = params.p;
    genA = params.getGeneratorA();
    genB = params.getGeneratorB();
    obits = params.orderLengthA;

    curve.calculateA24(params.p);

    // Sektion B
    r = curve.ladder3pt(genA[0], genA[1], genA[2], priv, obits, params.p);

    phiP = new MontgomeryPoint(genB[0], GF2Element.ONE(p));
    phiQ = new MontgomeryPoint(genB[1], GF2Element.ONE(p));
    phiD = new MontgomeryPoint(genB[2], GF2Element.ONE(p));

    fIso = new FourIsogeny(curve);
    fIso.calculatePM();
    fIso.calculateC4();

    // Sektion C
    maxIntPointsA = params.maxIntPointsA;
    maxA = params.maxA;
    splitsA = params.splitsA;
    points = new MontgomeryPoint[maxIntPointsA];
    pointsIndx = new int[maxIntPointsA];

    for (row = 1; row < maxA; row++) {
        while (index < maxA - row) {
            points[npts] = r;
            pointsIndx[npts++] = index;
            m = splitsA[ii++];
            r = fIso.multiply2X(r, 2*m);
            index += m;
        }

        fIso.getIsogeny(r);

        for (i = 0; i < npts; i++) {
            points[i] = fIso.imageByIsogeny(points[i]);
        }

        phiP = fIso.imageByIsogeny(phiP);
        phiQ = fIso.imageByIsogeny(phiQ);
        phiD = fIso.imageByIsogeny(phiD);

        r = points[npts-1];
        index = pointsIndx[npts-1];
        npts--;
    }

    // Sektion D
    fIso.getIsogeny(r);

    phiP = fIso.imageByIsogeny(phiP);
    phiQ = fIso.imageByIsogeny(phiQ);
    phiD = fIso.imageByIsogeny(phiD);

    inverses = GF2Element.threeInverts(phiP.z, phiQ.z, phiD.z);

    phiPX = inverses[0].multiply(phiP.x);
    phiQX = inverses[1].multiply(phiQ.x);
    phiDX = inverses[2].multiply(phiD.x);

    return new GF2Element[]{phiPX, phiQX, phiDX};
}

### Sektion A
Diese Sektion dient zum Abrufen der gewählten öffentlichen Parameter. Diese werden stets durch die voran definierte Klasse **PublicParameters** repräsentiert. Diese Sektion beinhaltet keine nennenswerten Berechnungen. Lediglich der für die Funktion **ladder3pt** (bzw. für die Funktion **doubleAdd**) notwendige Wert **a24** der initialen Montgomerykurve wird für diese berechnet.
### Sektion B
Innerhalb dieser Sektion wird zunächst der initiale Isogeniegeneratorpunkt durch die Linearkombination $P_A + [m]Q_A$ berechnet. Beide Punkte $P_A$ und $Q_A$ müssen daher die Ordnung $l_A^{e_A}$ besitzen, wobei $l_A^{e_A}$ der Grad der von Alice verwendeten Isogenie ist. In SIDHforJ verwendet Alice immer Isogenien der Form $4^n$. Also haben die Punkte $P_A$ und $Q_A$ jeweils eine Ordnung der Form $4^n$. 
Alice initialisiert schließlich eine Instanz der Klasse **FourIsogeny** und berechnet für diese die notwendigen Werte $a + 2c$ und $a - 2c$ (Diese vorab berechneten Werte werden in den Algorithmen zur Isogenieberechnung benötigt)
### Sektion C
In dieser Sektion berechnet Alice ihre geheime Isogenie, sowie die Bilder ihrer Generatorenpunkte durch diese Isogenie. Hierfür kommt der im Notebook "Theorie" besprochene "Kompositionsalgorithmus" zum Einsatz. Alice berechnet ihre geheime Isogenie $\phi_{Alice}$ von Grad $4^{e_A}$ nicht direkt, sondern teilt diese in $e_A$ Isogenien vom Grad $4$ auf. 
Innerhalb der **for-Schleife** wird also zunächst eine Isogenie vom Grad $4$ konstruiert und danach werden die Bilder der Generatorenpunkte durch eben diese Isogenie berechnet. Die Anzahl der Wiederholungen dieser **for-Schleife** wird einzig durch die von Alice verwendete Strategie (siehe Notebook "Theorie") bestimmt. 
### Sektion D
Die letze Wiederholung des "Kompositionsalgorithmus" wird außerhalb der **for-Schleife** durchgeführt. Hiernach sind die Bildpunkte **phiP, phiQ, phiD** durch die gesamte Isogenie (des Grads $4^{e_A}$) abgebildet worden. Diese werden aus Performancegründen (Reduktion der Schlüssellänge) in affine Koordinatenform gebracht und schließlich zurückgegeben.

Die exakt gleiche Methodik mit jeweils Isogenien vom Grad $3^{e_B}$ wird auch von Bob durchgeführt:

In [12]:
public GF2Element[] generatePubB(BigInteger priv, PublicParameters params) {
    // Sektion A
    MontgomeryCurve curve;
    GF2Element genA[], genB[], inverses[], phiPX, phiQX, phiDX;
    int maxIntPointsB, maxB, splitsB[], obits, row, m, index = 0, pointsIndx[], npts = 0, i, ii = 0;
    BigInteger p;
    MontgomeryPoint r, phiP, phiQ, phiD, points[];
    ThreeIsogeny threeIso;

    obits = params.orderLengthB;
    curve = params.curve;
    curve.calculateA24(params.p);
    p = params.p;

    genA = params.getGeneratorA();
    genB = params.getGeneratorB();

    // Sektion B
    r = curve.ladder3pt(genB[0], genB[1], genB[2], priv, obits, p);

    phiP = new MontgomeryPoint(genA[0], GF2Element.ONE(p));
    phiQ = new MontgomeryPoint(genA[1], GF2Element.ONE(p));
    phiD = new MontgomeryPoint(genA[2], GF2Element.ONE(p));

    threeIso = new ThreeIsogeny(curve);
    threeIso.calculatePM();

    // Sektion C
    maxIntPointsB = params.maxIntPointsB;
    maxB = params.maxB;
    splitsB = params.splitsB;
    points = new MontgomeryPoint[maxIntPointsB];
    pointsIndx = new int[maxIntPointsB];

    for (row = 1; row < maxB; row++) {
        while (index < maxB - row) {
            points[npts] = r;
            pointsIndx[npts++] = index;
            m = splitsB[ii++];
            r = threeIso.multiply3X(r, m);
            index += m;
        }


        threeIso.getIsogeny(r);

        for (i = 0; i < npts; i++) {
            points[i] = threeIso.imageByIsogeny(points[i]);
        }

        phiP = threeIso.imageByIsogeny(phiP);
        phiQ = threeIso.imageByIsogeny(phiQ);
        phiD = threeIso.imageByIsogeny(phiD);

        r = points[npts-1];
        index = pointsIndx[npts-1];
        npts--;

    }

    // Sektion D
    threeIso.getIsogeny(r);

    phiP = threeIso.imageByIsogeny(phiP);
    phiQ = threeIso.imageByIsogeny(phiQ);
    phiD = threeIso.imageByIsogeny(phiD);

    inverses = GF2Element.threeInverts(phiP.z, phiQ.z, phiD.z);

    phiPX = inverses[0].multiply(phiP.x);
    phiQX = inverses[1].multiply(phiQ.x);
    phiDX = inverses[2].multiply(phiD.x);

    return new GF2Element[]{phiPX, phiQX, phiDX};
}

## Berechnung des gemeinsamen Geheimnis

Schließlich benötigen wir nun noch die Funktionen zur Berechnung des gemeinsamen Geheimnis, nach beidseitigem Austausch der öffentlichen Schlüssel.

In [13]:
public byte[] generateSecretA(BigInteger privateA, GF2Element[] pubB, PublicParameters params) {
    // Sektion A
    GF2Element phiPB, phiQB, phiDB, rA, two;
    MontgomeryPoint points[], r;
    int pointsIndx[], row, index = 0, npts = 0, ii = 0, i, m;
    FourIsogeny fIso;

    phiPB = pubB[0];
    phiQB = pubB[1];
    phiDB = pubB[2];

    rA = MontgomeryCurve.recoverFromPoints(phiPB, phiQB, phiDB);
    fIso = new FourIsogeny(rA, phiPB.x0.p);
    fIso.calculateA24(params.p);

    two = GF2Element.ONE(phiDB.x0.p).shift(1);
    fIso.setaP2c(two.add(rA));
    fIso.setC4(two.shift(1));

    // Sektion B
    points = new MontgomeryPoint[params.maxIntPointsA];
    pointsIndx = new int[params.maxIntPointsA];

    r = fIso.ladder3pt(phiPB, phiQB, phiDB, privateA, params.orderLengthA, params.p);

    for (row = 1; row < params.maxA; row++) {
        while (index < params.maxA - row) {
            points[npts] = r;
            pointsIndx[npts++] = index;
            m = params.splitsA[ii++];
            r = fIso.multiply2X(r, 2*m);
            index += m;
        }

        fIso.getIsogeny(r);

        for (i = 0; i < npts; i++) {
            points[i] = fIso.imageByIsogeny(points[i]);
        }

        r = points[npts-1];
        index = pointsIndx[npts-1];
        npts--;
    }

    // Sektion C
    fIso.getIsogeny(r);
    fIso.calculateAC(4);

    return fIso.jInv().toByteArray();
}

Nach aufmerksamer Analyse der Funktion zur Berechnung der öffentlichen Schlüssel, sollte diese Funktion einfach zu verstehen sein.
### Sektion A
Die öffentlichen Schlüssel von Alice und von Bob werden zunächst gespeichert. Bobs Bildkurve wird durch die Funktion **recoverFromPoints** nach Eingabe von Bobs öffentlichen Punkten rekonstruiert.
Alice berechnet zudem die  für die Isogenieberechnungen notwendigen Werte $\frac{2a}{4}, a + 2c, 4c$
### Sektion B

Alice wendet nun ihre eigene geheime Isogenie auf Bobs Bildkurve an. Dies geschieht mit dem gleichen, voran vorgestellten "Kompositionsalgorithmus"

### Sektion C
Alice berechnet nun die letzte Wiederholung des "Kompositionsalgorithmus" außerhalb der **for-Schleife** und erhält nach Abschluss die Bildkurve von Bobs öffentlicher Kurve (Bobs Bildkurve) ihrer eigenen, geheimen Isogenie.
Da diese Isogenie durch die Klasse **FourIsogeny** implementiert wird und daher, wie bereits besprochen, die Bildkurve speichert, kann Sie diese schließlich in deren j-Invariante encodieren. 
Diese j-Invariante stellt das gemeinsame Geheimnis dar.

Bob führt die gleich Methodik auf Basis seiner Isogenien des Grad $3^n$ aus

In [14]:
public byte[] generateSecretB(BigInteger privateB, GF2Element[] pubA, PublicParameters params) {
    // Sektion A
    int i, ii = 0, row, m, index = 0, pointsIdx[], npts = 0;
    MontgomeryPoint r, points[];
    GF2Element phiPA, phiQA, phiDA, rA, temp;
    ThreeIsogeny threeIso;

    phiPA = pubA[0];
    phiQA = pubA[1];
    phiDA = pubA[2];
    
    // Sektion B
    points = new MontgomeryPoint[params.maxIntPointsB];
    pointsIdx = new int[params.maxIntPointsB];

    rA = MontgomeryCurve.recoverFromPoints(phiPA, phiQA, phiDA);

    threeIso = new ThreeIsogeny(rA, params.p);
    threeIso.calculateA24(params.p);
    threeIso.calculatePM();

    r = threeIso.ladder3pt(phiPA, phiQA, phiDA, privateB, params.orderLengthB, params.p);

    for (row = 1; row < params.maxB; row++) {
        while (index < params.maxB - row) {
            points[npts] = r;
            pointsIdx[npts++] = index;
            m = params.splitsB[ii++];
            r = threeIso.multiply3X(r, m);
            index += m;
        }

        threeIso.getIsogeny(r);

        for (i = 0; i < npts; i++) {
            points[i] = threeIso.imageByIsogeny(points[i]);
        }

        r = points[npts-1];
        index = pointsIdx[npts-1];
        npts--;
    }

    // Sektion C
    threeIso.getIsogeny(r);
    threeIso.calculateAC(3);

    return threeIso.jInv().toByteArray();
}

# Beispiel

Zum Schluss wollen wir den SIDHforJ testen. Dazu verwenden wir die NIST-Parameterklasse "P503" 

### Öffentliche Parameter

In [15]:
BigInteger lA, lB, f, p;
int eA, eB, maxPointsA, maxPointsB;
int[] splitsA, splitsB;
GF2Element aPx, aQx, aDx, bPx, bQx, bDx;

lA = BigInteger.valueOf(2);
lB = BigInteger.valueOf(3);
eA = 250;
eB = 159;
f = BigInteger.ONE;

p = (((lA.pow(eA)).multiply(lB.pow(eB))).multiply(f)).subtract(BigInteger.ONE);

aPx = new GF2Element(new BigInteger("1f6d52a7563bb9356b98a116a0ca9775dbb7382eb29e24e45299d8939959eaeeb47ff3113f60882d12103e4b8b8cd2b97da14657ae8c128be82209d2ddfca9", 16),
                          new BigInteger("2d44c3fad24e4cbddc8a2d9de336a92a9912ee6d09e2dd5c33ab26d60a268ac91f38e1af4c2d5bfa2b87dd55c8ca6019c6b0c08ed92b5aeb6c65a8e06e53e9", 16), p);
aQx = new GF2Element(new BigInteger("97453912e12f3daf32eeffd618bd93d3bbbf399137bd39858cadefae382e42d6e60a62fd62417ad61a14b60db26125273ec980981325d86e55c45e3bb46b1", 16),
                          new BigInteger("0", 16), p);
aDx = new GF2Element(new BigInteger("173775ecbec79c78fd1ed5fe36075aace1f53f8ffb97d2a7e80dfc2875e77ec72d1d4a99e13353ec9d147badd96126948a72b30bdd7cebad7b54f8ddb5cd06", 16),
                          new BigInteger("2eaa224ddda149bbbb9089d2b2c471d068eca203465ce97dbc1c8ed0ebb0ff90e4fbe7e266bba99cbae051797b4d35d28e36c1b1cb994aeeed1cb59fe5015", 16), p);
bPx = new GF2Element(new BigInteger("21b7098b640a01d88708b729837e870cff9df6d4df86d86a7409f41156cb5f7b8514822730940c9b51e0d9821b0a67dd7ed98b9793685fa2e22d6d89d66a4e", 16),
                          new BigInteger("2f37f575bebbc33851f75b7ab5d89fc3f07e4df3cc52349804b8d17a17000a42fc6c5734b9fcfde669730f3e8569ceb53821d3e8012f7f391f57364f402909", 16), p);
bQx = new GF2Element(new BigInteger("1e7d6ebceec9cfc47779affd696a88a971cdf3ec61e009df55caf4b6e01903b2cd1a12089c2ece106bdf745894c14d7e39b6997f70023e0a23b4b3787ef08f", 16),
                          new BigInteger("0", 16), p);
bDx = new GF2Element(new BigInteger("d4818d120a24abf48db51d129e6b1f24f4bbb2c16facc0c8c06323eeec2fa5b5e887e17226417b1907310bfe6784fdebbac8c2a9abbe753f52259a7b7d70e", 16),
                          new BigInteger("19e75f0f03312d22cbbf153747525d89e5155babb8bf0c130cb567ca532f69aaf57ea7682b9957021d90414433abbeedc233e9082185781c16724c8c356777", 16), p);

maxPointsA = 7;
maxPointsB = 8;

splitsA = new int[]{61, 32, 16, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1, 8, 4, 2, 1, 1, 2, 1, 1,
            4, 2, 1, 1, 2, 1, 1, 16, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1, 8, 4, 2, 1,
            1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1, 29, 16, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1,
            1, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1, 13, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2,
            1, 1, 2, 1, 1, 5, 4, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1};
splitsB = new int[]{71, 38, 21, 13, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1, 5, 4, 2, 1, 1, 2, 1,
            1, 2, 1, 1, 1, 9, 5, 3, 2, 1, 1, 1, 1, 2, 1, 1, 1, 4, 2, 1, 1, 1, 2, 1, 1, 17, 9,
            5, 3, 2, 1, 1, 1, 1, 2, 1, 1, 1, 4, 2, 1, 1, 1, 2, 1, 1, 8, 4, 2, 1, 1, 1, 2, 1,
            1, 4, 2, 1, 1, 2, 1, 1, 33, 17, 9, 5, 3, 2, 1, 1, 1, 1, 2, 1, 1, 1, 4, 2, 1, 1, 1,
            2, 1, 1, 8, 4, 2, 1, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1, 16, 8, 4, 2, 1, 1, 1, 2,
            1, 1, 4, 2, 1, 1, 2, 1, 1, 8, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 1};

In [16]:
PublicParameters P503 = new PublicParameters(f, lA, lB, eA, eB, new GF2Element[]{aPx, aQx, aDx}, new GF2Element[]{bPx, bQx, bDx}, splitsA, splitsB, maxPointsA, maxPointsB);

Die Primcharakteristik in Dezimaldarstellung:

In [17]:
P503.p

13175843156907117380839252916199345042492186767578363998445663477035843932020761233518914911546024351608607150390087656982982306331019593961154237431807

### Generierung der geheimen Schlüssel

Alice und Bob generieren sich nun beide eine eigene große und zufällige Zahl $k$. Diese Zahl muss jedoch insofern mit den Isogeniegraden kompatibel sein, als dass die Linearkombination der öffentlichen Generatorenpunkte mit Skalar $k$ wieder einen Punkt mit der geforderten Ordnung berechnet:

In [18]:
public class Random {
    public static BigInteger generateRandomOfOrder(BigInteger order) {
        SecureRandom rnd = new SecureRandom();
        int numBits = order.bitLength();
        BigInteger random = new BigInteger (numBits, rnd);
        while (random.compareTo(order) >= 0) random = new BigInteger (numBits, rnd);
        return random;
    }
}

In [19]:
public BigInteger generateRandom(int AorB, PublicParameters params) {
        boolean mask;
        BigInteger random, order, randomMod;
        if (AorB == 0) {
            order = params.orderA;
        } else {
            order = params.orderB;
        }
        random = Random.generateRandomOfOrder(order);
        if (AorB == 0) {
            mask = random.testBit(0);
        } else {
            randomMod = random.mod(BigInteger.valueOf(3));
            mask = randomMod.intValue() != 0;
        }
        while (random.equals(BigInteger.ZERO) || mask) {
            random = Random.generateRandomOfOrder(order);
            if (AorB == 0) {
                mask = random.testBit(0);
            } else {
                randomMod = random.mod(BigInteger.valueOf(3));
                mask = randomMod.intValue() != 0;
            }
        }
        return random;
}

In [20]:
BigInteger k_Alice, k_Bob;

k_Alice = generateRandom(0, P503);
k_Bob = generateRandom(1, P503);

System.out.println(k_Alice);
System.out.println(k_Bob);

1295673057904754938114711848654323712850732141922922917788605626889707218754
6581620086791301404265588030487809611609519149687558345655149062692800614087


### Berechnung der öffentlichen Schlüssel

Alice und Bob nutzen nun die vorab definierte Schnittstelle, um sich ihre jeweiligen öffentlichen Schlüssel zu berechnen

In [21]:
GF2Element[] pubA, pubB;

pubA = generatePubA(k_Alice, P503);

System.out.println(pubA[0]);
System.out.println(pubA[1]);
System.out.println(pubA[2]);

63856748500489114615191510543053201016614613774265323008450664723827057142997249649726372505580517672954339898724540049054065357467930945693741080576 + 1853657926857054907835570459798146502012718253312641365832270594803756137277869177162210283951520902744446473918380760113448921341189926563979101390638 * i {13175843156907117380839252916199345042492186767578363998445663477035843932020761233518914911546024351608607150390087656982982306331019593961154237431807}
4399334914562069859463705511818049674178931391769932662349140070170520599483016254067170886587890385503635857767860562599777604729049171583208682907977 + 5699239307532623193414562795367972264846207759297881806513014711942923866625157617442956260322860878238672409334743343677185187355083274117084424483412 * i {13175843156907117380839252916199345042492186767578363998445663477035843932020761233518914911546024351608607150390087656982982306331019593961154237431807}
639769721795046108760550569768671904452559990347646407429357085784259093

In [22]:
pubB = generatePubB(k_Bob, P503);

System.out.println(pubB[0]);
System.out.println(pubB[1]);
System.out.println(pubB[2]);

79263349441043752810076794135812974543651216972738776448365206043155735242597590751414408681420749326952575772710747193350034140653302818410847854197 + 2457314535888227480886214074622865450630171393926011042563152126163945054760452560279408903203665751147302593921315605382922993572145311153184016004664 * i {13175843156907117380839252916199345042492186767578363998445663477035843932020761233518914911546024351608607150390087656982982306331019593961154237431807}
4414785687906461291377556307761271763486432635743160770750893797607605174059711038108630125811983866463573566016541870433253111678149648607037843152342 + 10641075852245220616439041121256201553506007644190003081996667032325775708626652782126119089115288557664024538737520144047742194229053286108262507160019 * i {13175843156907117380839252916199345042492186767578363998445663477035843932020761233518914911546024351608607150390087656982982306331019593961154237431807}
79613749423313907254451265052252118054859329333308697801174700918358526

### Berechnung des gemeinsamen Geheimnis

Wie stellen uns nun vor, dass Alice und Bob ihre öffentlichen Schlüssel ausgetaucht haben. Nun berechnen Sie ihr gemeinsames Geheimnis mit der voran definierten Schnittstelle

In [23]:
byte[] sharedA, sharedB;

sharedA = generateSecretA(k_Alice, pubB, P503);
sharedB = generateSecretB(k_Bob, pubA, P503);

Nun ist der alles entscheidende Moment gekommen: Stimmen die beiden j-Invarianten überein?

In [24]:
Arrays.toString(sharedA)

[4, 65, -47, 22, -103, -121, 86, -14, 67, -50, -50, 19, -40, 111, 72, 39, 83, 88, 46, 44, -64, 98, -35, -128, 33, -3, 92, 100, 62, -42, 63, 3, 127, -70, 73, 56, -13, 49, -126, -94, -6, 46, -1, -75, -15, -112, -94, -111, -124, 45, -6, 120, -62, -93, -57, 121, -31, -118, 121, 55, 42, 2, 118, 48, -44, 32, 58, 28, 57, -103, -25, 115, -43, -111, 59, 109, 84, -64, 59, -111, -92, -18, 113, 21, 102, 116, -97, -26, -1, -1, -124, -55, 23, -15, 117, -75, 115, -2, 45, -8, -59, 88, 26, -95, 105, -32, 117, 102, 72, -11, 100, 10, 13, 87, -34, 3, 32, 101, 97, -73, -12, -36, 118, -60, -48, -87]

In [25]:
Arrays.toString(sharedB)

[4, 65, -47, 22, -103, -121, 86, -14, 67, -50, -50, 19, -40, 111, 72, 39, 83, 88, 46, 44, -64, 98, -35, -128, 33, -3, 92, 100, 62, -42, 63, 3, 127, -70, 73, 56, -13, 49, -126, -94, -6, 46, -1, -75, -15, -112, -94, -111, -124, 45, -6, 120, -62, -93, -57, 121, -31, -118, 121, 55, 42, 2, 118, 48, -44, 32, 58, 28, 57, -103, -25, 115, -43, -111, 59, 109, 84, -64, 59, -111, -92, -18, 113, 21, 102, 116, -97, -26, -1, -1, -124, -55, 23, -15, 117, -75, 115, -2, 45, -8, -59, 88, 26, -95, 105, -32, 117, 102, 72, -11, 100, 10, 13, 87, -34, 3, 32, 101, 97, -73, -12, -36, 118, -60, -48, -87]

# Referenzen
_[1]_ Class Long; https://docs.oracle.com/javase/7/docs/api/java/lang/Long.html; 2020 Oracle \
_[2]_ Efficient Algorithms for Supersingular Isogeny Diffie-Hellman; Longa Patrick, Naehrig Michael; 2016 Advances in Cryptology - CRYPTO 2016 \
_[3]_ Towards Quantum-Resistant Cryptosystems from Supersingular Elliptic Curve Isogenies; Jao David, De Feo Luca; 2011 PQCrypto 2011\
_[4]_ Public-Key Cryptosystem Based on Isogenies; Rostovtsev Alexander, Stolbunov Anton; 2006 DBLP \
_[5]_ Mathematics of Isogeny Based Cryptography; De Feo Luca; 2017 Université de Versailles

***
&copy; 2021, Linus Meierhöfer